In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.datasets import load_breast_cancer
import warnings
warnings.filterwarnings('ignore')

/anaconda3/envs/py37/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
train = pd.read_pickle('../input/train.pkl')
test = pd.read_pickle('../input/test.pkl')

In [6]:
target = train['isFraud']
train = train.drop(['isFraud','TransactionDT'],axis=1)

In [10]:
splits = 5
folds = KFold(n_splits = splits)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("Fold {}".format(fold_))
    print(trn_idx,len(trn_idx))
    print(val_idx,len(val_idx))

Fold 0
[118108 118109 118110 ... 590537 590538 590539] 472432
[     0      1      2 ... 118105 118106 118107] 118108
Fold 1
[     0      1      2 ... 590537 590538 590539] 472432
[118108 118109 118110 ... 236213 236214 236215] 118108
Fold 2
[     0      1      2 ... 590537 590538 590539] 472432
[236216 236217 236218 ... 354321 354322 354323] 118108
Fold 3
[     0      1      2 ... 590537 590538 590539] 472432
[354324 354325 354326 ... 472429 472430 472431] 118108
Fold 4
[     0      1      2 ... 472429 472430 472431] 472432
[472432 472433 472434 ... 590537 590538 590539] 118108


In [33]:
test_split = 5
X_test_index = np.array_split(test.index,test_split)
for i in range(test_split):
    X_test = test.loc[X_test_index[i]]

In [2]:
cancer = load_breast_cancer()

In [3]:
# cancer

In [4]:
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

In [5]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [6]:
params = {
        'objective': 'binary',
        "boosting_type": "gbdt",
        "metric": 'auc',
        'random_state': 47
    }

In [7]:
target = df['target']
x = df.drop('target',axis=1)
X_train,X_test,y_train,y_test = train_test_split(x,target,test_size = 0.5)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

test_split = 5
X_test_index = np.array_split(X_test.index,test_split)
# for i in range(test_split):
#     XX_train,X_val,yy_train,y_val = train_test_split(X_train,y_train,test_size = 0.2)
#     XX_test = X_test.loc[X_test_index[i]]
#     trn_data = lgb.Dataset(XX_train, label=yy_train)
#     val_data = lgb.Dataset(X_val, label=y_val)

#     lgb_model = lgb.train(params,
#                     trn_data,
#                     10000,
#                     valid_sets = [trn_data, val_data],
#                     verbose_eval=500,
#                     early_stopping_rounds=500)
#     pred = lgb_model.predict(XX_test)
#     df_pred = pd.DataFrame({'target':pred}, index=XX_test.index)
#     df_pred.loc[df_pred['target'] >0.5,'target'] = 1
#     df_pred.loc[df_pred['target'] <=0.5,'target'] = 0
#     X_train = pd.concat([X_train,XX_test])
#     y_train = pd.concat([y_train,df_pred['target']])
#     X_train = X_train[len(XX_test):]
#     y_train = y_train[len(XX_test):]

In [8]:
X_test_index

[Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
             34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
             51, 52, 53, 54, 55, 56],
            dtype='int64'),
 Int64Index([ 57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
              70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
              83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
              96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
             109, 110, 111, 112, 113],
            dtype='int64'),
 Int64Index([114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
             127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
             140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
             153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,


In [159]:
dd

,aa
